## AuxTel test LTS-337-015 (Small Slew and Settle)

In this notebook, we check the speed and accuracy of the slew and settle of the telescope with a
change of 10 degrees.  Desired specs:

| Description | Value       | Unit          |   Name     |
| :---        |    :----:   |       :----:  |       ---: |
|The pointing accuracy requirement for small slews is:    | 2.5       | Arcsec RMS   |Aux_Tel_Sm_Slew_Abs_Pointing|
The small elevation slew requirement for the Auxiliary Telescope used to define the Aux_Tel_Sm_Slew_Abs_Pointing requirement| 10| Degrees |Aux_Tel_Sm_Elev_Slew|
|The small azimuth slew requirement for the Auxiliary Telescope used to define the Aux_Tel_Sm_Slew_Abs_Pointing requirement.  |10       | Degrees     |Aux_Tel_Sm_Az_Slew|
|The allotted time to perform a slew and settle by Aux_Tel_Sm_Elev_Slew and Aux_Tel_Sm_Az_Slew by the Auxiliary Telescope and meet the Aux_Tel_Sm_Slew_Abs_Pointing pointing requirement.      | 10|Seconds|Aux_Tel_Smslew_AcqTime|

In [1]:
import sys, time, os, asyncio

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u
from lsst_efd_client import EfdClient

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.4.1/lib/python3.8/site-packages/jose/backends/cryptography_backend.py:23: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes, int_to_bytes


In [2]:
# Set Cerro Pachon location
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [3]:
# for tab completion to work in current notebook instance
%config IPCompleter.use_jedi = False

In [4]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG
# Make matplotlib less chatty
logging.getLogger("matplotlib").setLevel(logging.WARNING)

In [5]:
# Get EFD client and bring in Lupton's unpacking code
client = EfdClient('summit_efd')

def merge_packed_time_series(packed_dataframe, base_field, stride=1, 
                             ref_timestamp_col="cRIO_timestamp", internal_time_scale="tai"):
    """Select fields that are time samples and unpack them into a dataframe.
            Parameters
            ----------
            packedDF : `pandas.DataFrame`
                packed data frame containing the desired data
            base_field :  `str`
                Base field name that will be expanded to query all
                vector entries.
            stride : `int`, optional
                Only use every stride value when unpacking.  Must be a factor
                of the number of packed values.
                (1 by default)
            ref_timestamp_col : `str`, optional
                Name of the field name to use to assign timestamps to unpacked
                vector fields (default is 'cRIO_timestamp').
            internal_time_scale : `str`, optional
                Time scale to use when converting times to internal formats
                ('tai' by default). Equivalent to EfdClient.internal_scale
        Returns
            -------
            result : `pandas.DataFrame`
                A `pandas.DataFrame` containing the results of the query.
            """
    
    packed_fields = [k for k in packed_dataframe.keys() if k.startswith(base_field)]
    packed_fields = sorted(packed_fields, key=lambda k: int(k[len(base_field):]))  # sort by pack ID
    npack = len(packed_fields)
    if npack%stride != 0:
        raise RuntimeError(f"Stride must be a factor of the number of packed fields: {stride} v. {npack}")
    packed_len = len(packed_dataframe)
    n_used = npack//stride   # number of raw fields being used
    output = np.empty(n_used*packed_len)
    times = np.empty_like(output, dtype=packed_dataframe[ref_timestamp_col][0])
    
    if packed_len == 1:
        dt = 0
    else:
        dt = (packed_dataframe[ref_timestamp_col][1] - packed_dataframe[ref_timestamp_col][0])/npack
    for i in range(0, npack, stride):
        i0 = i//stride
        output[i0::n_used] = packed_dataframe[f"{base_field}{i}"]
        times[i0::n_used] = packed_dataframe[ref_timestamp_col] + i*dt
     
    timestamps = Time(times, format='unix', scale=internal_time_scale).datetime64
    return pd.DataFrame({base_field:output, "times":times}, index=timestamps)

Starting new HTTPS connection (1): roundtable.lsst.codes:443
https://roundtable.lsst.codes:443 "GET /segwarides/ HTTP/1.1" 200 253
Starting new HTTPS connection (1): roundtable.lsst.codes:443
https://roundtable.lsst.codes:443 "GET /segwarides/creds/summit_efd HTTP/1.1" 200 92
Starting new HTTPS connection (1): influxdb-summit-efd.lsst.codes:443
https://influxdb-summit-efd.lsst.codes:443 "GET /health HTTP/1.1" 200 106


In [6]:
#get classes and start them
domain = salobj.Domain()
await asyncio.sleep(10) # This can be removed in the future...
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

atmcs: Adding all resources.
atptg: Adding all resources.
ataos: Adding all resources.
atpneumatics: Adding all resources.
athexapod: Adding all resources.
atdome: Adding all resources.
atdometrajectory: Adding all resources.
atcamera: Adding all resources.
atspectrograph: Adding all resources.
atheaderservice: Adding all resources.
atarchiver: Adding all resources.
Read historical data in 0.01 sec
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, algorithm)
Read 28 history items for RemoteEvent(ATDomeTrajectory, 0, appliedSettingsMatchStart)
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, authList)
Read 100 history items for RemoteEvent(ATDomeTrajectory, 0, heartbeat)
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, logLevel)
Read 100 history items for RemoteEvent(ATDomeTrajectory, 0, logMessage)
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, settingVersions)
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, settingsApplied)
Read 1 history

[[None, None, None, None, None, None, None], [None, None, None, None]]

mountStatus DDS read queue is full (100 elements); data may be lost
guidingAndOffsets DDS read queue is full (100 elements); data may be lost
currentTargetStatus DDS read queue is full (100 elements); data may be lost


In [51]:
# enable components if required
await atcs.enable()
await latiss.enable()

Enabling all components
Gathering settings.
Couldn't get settingVersions event. Using empty settings.
Complete settings for atmcs.
Complete settings for atptg.
Complete settings for ataos.
Complete settings for atpneumatics.
Complete settings for athexapod.
Complete settings for atdome.
Complete settings for atdometrajectory.
Settings versions: {'atmcs': '                                                                                                                               ', 'atptg': '', 'ataos': 'current', 'atpneumatics': '                                                                                                                               ', 'athexapod': 'summit', 'atdome': 'test', 'atdometrajectory': ''}
[atmcs]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atptg]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[ataos]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atpneumatics]::[<State.STANDBY: 5>, <State.DISABLED: 

In [ ]:
# All components enabled 1:02 PM

In [8]:
# take event checking out the slew commands to test telescope only
# otherwise it'll wait for the dome before completing slew command
atcs.check.atdome = False
atcs.check.atdometrajectory = False

In [9]:
# turn on ATAOS corrections just to make sure the mirror is under air
tmp = await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=False)

In [10]:
# Ensure we're using Nasmyth 2
await atcs.rem.atptg.cmd_focusName.set_start(focus=3)

In [38]:
# point telescope to desired starting position
start_az=0
start_el=70
start_rot_pa=0
await atcs.point_azel(start_az, start_el, rot_tel=start_rot_pa, wait_dome=False)

Sending command
Stop tracking.
Unknown tracking state: 10
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
[Telescope] delta Alt = -000.006 deg; delta Az= +000.001 deg; delta N1 = +000.000 deg; delta N2 = -035.349 deg 
[Telescope] delta Alt = +000.000 deg; delta Az= +000.001 deg; delta N1 = +000.000 deg; delta N2 = -029.549 deg 
[Telescope] delta Alt = +000.000 deg; delta Az= +000.000 deg; delta N1 = +000.000 deg; delta N2 = -023.600 deg 
[Telescope] delta Alt = -000.000 deg; delta Az= -000.000 deg; delta N1 = +000.000 deg; delta N2 = -017.809 deg 
[Telescope] delta Alt = -000.000 deg; delta Az= -000.000 deg; delta N1 = +000.000 deg; delta N2 = -014.142 deg 
[Telescope] delta Alt = -000.000 deg; delta Az= -000.000 deg; delta N1 = +000.000 deg; delta N2 = -010.745 deg 
[Telescope] delta Alt = -000.000 deg; delta Az= -000.000 deg; delta N1 = +000

In [39]:
#declare offset sizes for tests

# For small slew and settle
d_az= 10 # degrees
d_el = 10 # degrees
d_rot= 0 # degrees

"""
# For large slew and settle
d_az= 90 # degrees
d_el = 60 # degrees
d_rot= 0 # degrees

# What we actually care about for the survey! So measure this too
d_az= 3.5 # degrees
d_el = 3.5 # degrees
d_rot= 0 # degrees
"""

'\n# For large slew and settle\nd_az= 90 # degrees\nd_el = 60 # degrees\nd_rot= 0 # degrees\n\n# What we actually care about for the survey! So measure this too\nd_az= 3.5 # degrees\nd_el = 3.5 # degrees\nd_rot= 0 # degrees\n'

In [40]:
# get RA/DEC of current telescope Alt/Az position
az = Angle(start_az, unit=u.deg)
el = Angle(start_el, unit=u.deg)
print(f'orig az {az} and el {el}')
time_data = await atcs.rem.atptg.tel_timeAndDate.next(flush=True, timeout=2)
# This should be TAI and not UTC... so will be 37s off system clock seconds ??
curr_time_atptg = Time(time_data.mjd, format="mjd")
coord_frame_AltAz = AltAz(location=location, obstime=curr_time_atptg)
coord_frame_radec = ICRS()
coord_azel = AltAz(az=az, alt=el, location=location, obstime=curr_time_atptg)
ra_dec = coord_azel.transform_to(coord_frame_radec)
print('Current Position is: \n {}'.format(coord_azel))
print('Current Position is: \n {}'.format(ra_dec))

# get RA/DEC of target position
az = Angle(start_az+d_az, unit=u.deg)
el = Angle(start_el+d_el, unit=u.deg)
print(f'target az {az} and el {el}')
coord_azel_target = AltAz(az=az, alt=el, location=location, obstime=curr_time_atptg)
ra_dec_target = coord_azel_target.transform_to(coord_frame_radec)
print('Target Position is: \n {}'.format(coord_azel_target))
print('Target Position is: \n {}'.format(ra_dec_target))

orig az 0.0 deg and el 70.0 deg
Current Position is: 
 <AltAz Coordinate (obstime=59278.7058903381, location=(1819093.56876225, -5208411.6827961, -3195180.61110659) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (0., 70.)>
Current Position is: 
 <ICRS Coordinate: (ra, dec) in deg
    (346.76145318, -10.35528365)>
target az 10.0 deg and el 80.0 deg
Target Position is: 
 <AltAz Coordinate (obstime=59278.7058903381, location=(1819093.56876225, -5208411.6827961, -3195180.61110659) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (10., 80.)>
Target Position is: 
 <ICRS Coordinate: (ra, dec) in deg
    (348.60129698, -20.49533785)>


In [27]:
# Slew to starting position and take an image to check headers
await atcs.slew_icrs(ra=str(ra_dec.ra), dec=str(ra_dec.dec), rot=0.0,
                      slew_timeout=240., stop_before_slew=False, wait_settle=False)


print('track for 2s')
await asyncio.sleep(2)
# take a quick image to get some header data
await latiss.take_engtest(exptime=2)


Auto sky angle: 0.0 deg
Sending command
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
Got False
Telescope not in position
[Telescope] delta Alt = +000.094 deg; delta Az= +000.275 deg; delta N1 = -000.000 deg; delta N2 = +069.478 deg 
[Telescope] delta Alt = -000.000 deg; delta Az= +000.002 deg; delta N1 = -000.000 deg; delta N2 = +066.315 deg 
[Telescope] delta Alt = -000.000 deg; delta Az= +000.007 deg; delta N1 = -000.000 deg; delta N2 = +062.334 deg 
[Telescope] delta Alt = -000.000 deg; delta Az= +000.006 deg; delta N1 = -000.000 deg; delta N2 = +058.356 deg 
[Telescope] delta Alt = +000.000 deg; delta Az= +000.006 deg; delta N1 = -000.000 deg; delta N2 = +054.382 deg 
[Telescope] delta Alt = +000.000 deg; delta Az= +000.007 deg; delta N1 = -000.000 deg; delta N2 = +048.399 deg 
[Telescope] delta Alt = -000.000 deg; delta Az= +000.007 de

array([2021030500105])

In [41]:
# This is where the tests are carried out.
times = []
ra_rms_accuracies = []
dec_rms_accuracies = []
ra_mean_accuracies = []
dec_mean_accuracies = []

# Slew to starting position
await atcs.slew_icrs(ra=str(ra_dec.ra), dec=str(ra_dec.dec), rot=0.0,
                      slew_timeout=240., stop_before_slew=False, wait_settle=False)


print('track for 2s')
await asyncio.sleep(2)
# take an image
await latiss.take_engtest(exptime=2)

# Now slew to new target
print('Starting to Slew to target')
start_time = Time(Time.now(), format='fits', scale='tai')
await atcs.slew_icrs(ra=str(ra_dec_target.ra), dec=str(ra_dec_target.dec),
                      rot=d_rot, slew_timeout=240., 
                      stop_before_slew=False, wait_settle=False)
end_time = Time(Time.now(), format='fits', scale='tai')
slew_time = TimeDelta(end_time - start_time, format='sec').value
print(f'Time to slew is {slew_time} seconds')
times.append(slew_time)
await latiss.take_engtest(exptime=2)
await asyncio.sleep(5)
t_end = end_time + TimeDelta(5, format='sec')
nsec = 2
mount_position = await client.select_time_series("lsst.sal.ATPtg.mount_positions", ['*'],
                                          t_end - TimeDelta(nsec, format='sec'), t_end)
ra = merge_packed_time_series(mount_position, 'ra', stride=1)
dec = merge_packed_time_series(mount_position, 'declination', stride=1)
ra_vals = np.array(ra.values.tolist())[:,0]
dec_vals = np.array(dec.values.tolist())[:,0]

ra_mean = np.mean(ra_vals)
dec_mean = np.mean(dec_vals)
# Errors in arcsec
ra_rms = np.sqrt(np.mean((ra_vals - ra_mean)**2)) * 3600.0 
dec_rms = np.sqrt(np.mean((dec_vals - dec_mean)**2)) * 3600.0 
ra_mean_error = abs(ra_dec.ra.value - ra_mean) * 3600
dec_mean_error = abs(ra_dec.dec.value - dec_mean) * 3600
print(f"RA rms error= {ra_rms} arcseconds. Dec rms error= {dec_rms} arcseconds.")
print(f"RA mean error = {ra_mean_error} arcseconds. Dec mean error = {dec_mean_error} arcseconds.")

ra_rms_accuracies.append(ra_rms)
dec_rms_accuracies.append(dec_rms)
ra_mean_accuracies.append(ra_mean_error)
dec_mean_accuracies.append(dec_mean_error)

# Repeat this going back to origin

print('Starting to Slew to target')
start_time = Time(Time.now(), format='fits', scale='tai')
await atcs.slew_icrs(ra=str(ra_dec.ra), dec=str(ra_dec.dec), rot=0.0,
                      slew_timeout=240., stop_before_slew=False, wait_settle=False)

end_time = Time(Time.now(), format='fits', scale='tai')
slew_time = TimeDelta(end_time - start_time, format='sec').value
print(f'Time to slew is {slew_time} seconds')
times.append(slew_time)
await latiss.take_engtest(exptime=2)
await asyncio.sleep(5)
t_end = end_time + TimeDelta(5, format='sec')
nsec = 2
mount_position = await client.select_time_series("lsst.sal.ATPtg.mount_positions", ['*'],
                                          t_end - TimeDelta(nsec, format='sec'), t_end)
ra = merge_packed_time_series(mount_position, 'ra', stride=1)
dec = merge_packed_time_series(mount_position, 'declination', stride=1)
ra_vals = np.array(ra.values.tolist())[:,0]
dec_vals = np.array(dec.values.tolist())[:,0]
ra_mean = np.mean(ra_vals)
dec_mean = np.mean(dec_vals)
# Errors in arcsec
ra_rms = np.sqrt(np.mean((ra_vals - ra_mean)**2)) * 3600.0 
dec_rms = np.sqrt(np.mean((dec_vals - dec_mean)**2)) * 3600.0 
ra_mean_error = abs(ra_dec.ra.value - ra_mean) * 3600
dec_mean_error = abs(ra_dec.dec.value - dec_mean) * 3600
print(f"RA rms error= {ra_rms} arcseconds. Dec rms error= {dec_rms} arcseconds.")
print(f"RA mean error = {ra_mean_error} arcseconds. Dec mean error = {dec_mean_error} arcseconds.")

ra_rms_accuracies.append(ra_rms)
dec_rms_accuracies.append(dec_rms)
ra_mean_accuracies.append(ra_mean_error)
dec_mean_accuracies.append(dec_mean_error)


Auto sky angle: 0.0 deg
Sending command
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
Got False
Telescope not in position
[Telescope] delta Alt = +000.093 deg; delta Az= +000.331 deg; delta N1 = -000.000 deg; delta N2 = +069.202 deg 
[Telescope] delta Alt = -000.000 deg; delta Az= +000.004 deg; delta N1 = -000.000 deg; delta N2 = +065.698 deg 
[Telescope] delta Alt = -000.000 deg; delta Az= +000.007 deg; delta N1 = -000.000 deg; delta N2 = +061.725 deg 
[Telescope] delta Alt = +000.000 deg; delta Az= +000.007 deg; delta N1 = -000.000 deg; delta N2 = +057.737 deg 
[Telescope] delta Alt = +000.000 deg; delta Az= +000.006 deg; delta N1 = -000.000 deg; delta N2 = +051.761 deg 
[Telescope] delta Alt = +000.000 deg; delta Az= +000.006 deg; delta N1 = +000.000 deg; delta N2 = +045.794 deg 
[Telescope] delta Alt = -000.000 deg; delta Az= +000.006 de

In [42]:
# Now check to see if the specs are met:
Aux_Tel_Smslew_AcqTime = 10.0
if max(times) < Aux_Tel_Smslew_AcqTime:
    print(f"Aux_Tel_Smslew_AcqTime passed.  Spec = {Aux_Tel_Smslew_AcqTime}.\
    Measured = {max(times)} ")
else:
    print(f"Aux_Tel_Smslew_AcqTime failed!  Spec = {Aux_Tel_Smslew_AcqTime}.\
    Measured = {max(times)} ")

Aux_Tel_Sm_Slew_Abs_Pointing = 2.5
# First look at the RMS accuracies
worst_case = max(ra_rms_accuracies + dec_rms_accuracies)
if worst_case < Aux_Tel_Sm_Slew_Abs_Pointing:
    print(f"Aux_Tel_Sm_Slew_Abs_Pointing (RMS only) passed.  Spec = {Aux_Tel_Sm_Slew_Abs_Pointing}. \
    Measured = {worst_case} ")
else:
    print(f"Aux_Tel_Sm_Slew_Abs_Pointing (RMS only) failed!  Spec = {Aux_Tel_Sm_Slew_Abs_Pointing}. \
    Measured = {worst_case} ")

# Next look at the mean offset.  this will likely fail due to timebase errors
worst_case = max(ra_mean_accuracies + dec_mean_accuracies)
if worst_case < Aux_Tel_Sm_Slew_Abs_Pointing:
    print(f"Aux_Tel_Sm_Slew_Abs_Pointing (mean offset) passed.  Spec = {Aux_Tel_Sm_Slew_Abs_Pointing}. \
    Measured = {worst_case} ")
else:
    print(f"Aux_Tel_Sm_Slew_Abs_Pointing (mean offset) failed!  Spec = {Aux_Tel_Sm_Slew_Abs_Pointing}. \
    Measured = {worst_case} ")


Aux_Tel_Smslew_AcqTime failed!  Spec = 10.0.    Measured = 10.388086000003227 
Aux_Tel_Sm_Slew_Abs_Pointing (RMS only) passed.  Spec = 2.5.     Measured = 0.1005134760135141 
Aux_Tel_Sm_Slew_Abs_Pointing (mean offset) failed!  Spec = 2.5.     Measured = 36503.492271020434 


In [57]:
# For shutdown of system
await atcs.stop_tracking()

Stop tracking.


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=2091141490, ack=<SalRetCode.CMD_FAILED: -302>, error=6612, result='Rejected : command not allowed in current state')

In [45]:
# turn off corrections
tmp = await atcs.rem.ataos.cmd_disableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [52]:
# take event checking out the slew commands to test telescope only
# otherwise it'll wait for the dome before completing slew command
atcs.check.atdome = True
atcs.check.atdometrajectory = True

In [53]:
# Putting everything back in standby.
await atcs.shutdown()

Disabling ATAOS corrections
Disabling ATAOS corrections.
Closing M1 cover vent gates.
Cover state <MirrorCoverState.CLOSED: 6>
M1 cover already closed.
M1 vent state <VentsPosition.CLOSED: 1>
M1 vents already closed.
Close dome.
ATDome Shutter Door is already closed. Ignoring.
Slew dome to Park position.
Sending ATDomeTrajectory to DISABED state. Component will be left in DISABLEDstate or else it may send the ATDome back to alignment with the telescope.
process as completed...
atdometrajectory: <State.DISABLED: 1>
ATDome in position.
[Dome] delta Az = +002.960 deg
Axes in position.
Disable ATDomeTrajectory
Slew telescope to Park position.
Sending command
Stop tracking.
Unknown tracking state: 10
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
[Telescope] delta Alt = -000.025 deg; delta Az= +000.001 deg; delta N1 = -000.000 deg; delta N2 = -044

RuntimeError: Failed to transition ['atdome'] to <State.STANDBY: 5>.

In [54]:
await atcs.rem.atdome.cmd_start.set_start(settingsToApply="test", timeout=30)

In [55]:
await salobj.set_summary_state(atcs.rem.atdome, salobj.State.STANDBY, settingsToApply="test")

[<State.DISABLED: 1>, <State.STANDBY: 5>]

In [48]:
await salobj.set_summary_state(atcs.rem.atdometrajectory, salobj.State.ENABLED)

[<State.DISABLED: 1>, <State.ENABLED: 2>]

In [62]:
await salobj.set_summary_state(latiss.rem.atspectrograph, salobj.State.STANDBY)
await salobj.set_summary_state(latiss.rem.atcamera, salobj.State.STANDBY)
await salobj.set_summary_state(latiss.rem.atheaderservice, salobj.State.STANDBY)
await salobj.set_summary_state(latiss.rem.atarchiver, salobj.State.STANDBY)

[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]

In [60]:
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.STANDBY)

[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]

In [61]:
await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.STANDBY)

[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]